In [2]:
import pygame
import math
import random

# Define constants
WIDTH, HEIGHT = 800, 600
GRAVITATIONAL_CONSTANT = 0.1
SPLIT_PARTICLE_PERCENTAGE = 0.1

# Define the Particle class
class Particle:
    def __init__(self, x, y, mass, color):
        self.x = x
        self.y = y
        self.mass = mass
        self.vx = random.uniform(-1, 1)
        self.vy = random.uniform(-1, 1)
        self.color = color

    def apply_force(self, force):
        ax = force[0] / self.mass
        ay = force[1] / self.mass
        self.vx += ax
        self.vy += ay

    def update(self):
        self.x += self.vx
        self.y += self.vy

# Initialize pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()

# Function to calculate gravitational force between particles
def calculate_gravitational_force(particle1, particle2):
    dx = particle2.x - particle1.x
    dy = particle2.y - particle1.y
    distance = math.sqrt(dx**2 + dy**2)
    force_magnitude = (GRAVITATIONAL_CONSTANT * particle1.mass * particle2.mass) / (distance**2)
    angle = math.atan2(dy, dx)
    force_x = force_magnitude * math.cos(angle)
    force_y = force_magnitude * math.sin(angle)
    return force_x, force_y

# Function to check collision between particles
def check_collision(particle1, particle2):
    dx = particle2.x - particle1.x
    dy = particle2.y - particle1.y
    distance = math.sqrt(dx**2 + dy**2)
    if distance < 10:  # Adjust the collision radius as needed
        return True
    return False

# Function to handle merging particles and splitting with additional condition
def merge_and_split(particle1, particle2):
    if min(particle1.mass, particle2.mass) < 0.33 * max(particle1.mass, particle2.mass):
        merged_mass = particle1.mass + particle2.mass
        merged_particle = Particle(
            (particle1.x + particle2.x) / 2,
            (particle1.y + particle2.y) / 2,
            merged_mass,
            (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        )
        return [merged_particle]
    else:
        merged_mass = particle1.mass + particle2.mass
        merged_particle = Particle(
            (particle1.x + particle2.x) / 2,
            (particle1.y + particle2.y) / 2,
            merged_mass,
            (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        )
        num_split_particles = random.randint(3, 5)
        total_mass = merged_particle.mass * SPLIT_PARTICLE_PERCENTAGE
        new_particles = []
        for _ in range(num_split_particles):
            particle_mass = total_mass * random.random()
            direction = random.uniform(0, 2 * math.pi)
            new_particle = Particle(
                merged_particle.x,
                merged_particle.y,
                particle_mass,
                merged_particle.color
            )
            new_particle.vx = math.cos(direction) * 2
            new_particle.vy = math.sin(direction) * 2
            new_particles.append(new_particle)
        return new_particles

# User input for number of particles and their mass
num_particles = int(input("Enter the number of particles: "))
particles = []
for i in range(num_particles):
    x = random.uniform(50, WIDTH - 50)
    y = random.uniform(50, HEIGHT - 50)
    particle_mass = float(input(f"Enter the mass of particle {i + 1}: "))
    particle_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
    particle = Particle(x, y, particle_mass, particle_color)
    particles.append(particle)

# Main loop
running = True
while running:
    screen.fill((255, 255, 255))

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Calculate forces between particles
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            force = calculate_gravitational_force(particles[i], particles[j])
            particles[i].apply_force(force)
            particles[j].apply_force((-force[0], -force[1]))  # Newton's third law

    # Check for collisions and handle them
    for i in range(num_particles):
        for j in range(i + 1, num_particles):
            if check_collision(particles[i], particles[j]):
                new_particles = merge_and_split(particles[i], particles[j])
                particles.pop(j)
                particles.pop(i)
                particles.extend(new_particles)
                num_particles = len(particles)
                break
        else:
            continue
        break

    # Update particle positions and draw them
    for particle in particles:
        particle.update()
        # Keep particles within the view
        if particle.x < 0 or particle.x > WIDTH:
            particle.vx *= -1
        if particle.y < 0 or particle.y > HEIGHT:
            particle.vy *= -1
        pygame.draw.circle(screen, particle.color, (int(particle.x), int(particle.y)), 5)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


Enter the number of particles:  10
Enter the mass of particle 1:  11
Enter the mass of particle 2:  56
Enter the mass of particle 3:  99
Enter the mass of particle 4:  45
Enter the mass of particle 5:  26
Enter the mass of particle 6:  74
Enter the mass of particle 7:  12
Enter the mass of particle 8:  32
Enter the mass of particle 9:  19
Enter the mass of particle 10:  43
